There are `n` children standing in a line. Each child is assigned a rating value given in the integer array ratings.

You are giving candies to these children subjected to the following requirements:

* Each child must have at least one candy.
* Children with a higher rating get more candies than their neighbors.

Return the minimum number of candies you need to have to distribute the candies to the children.

---

We can take an approach where we linearly probe the array, consecutively adding candies to children who have higher ratings than their neighbor but a not-higher candy count.

On each full probe, we reduce the search space by 2 (the first and last positions are optimal at each pass). We store candy amounts in an intermediate array.

This implies $O(n^2)$ time complexity, $O(n)$ space complexity.

In [10]:
def candy(ratings: list[int]) -> int:
    n = len(ratings)
    candies = [1] * n
    print(candies)
    
    passes = 0
    unchecked = n
    while unchecked > 0:
        # if unchecked == 1:
        #     candies[n \\ 2]
        for i in range(passes + 1, n - passes - 1):
            # Check to the left and right of current index:
            if candies[i] <= candies[i-1] or candies[i] <= candies[i+1]:
                candies[i] += 1
        print(f"{candies}, {unchecked}, {passes}")
        unchecked -= 2


In [11]:
ratings = [1, 2, 3, 4, 3]
candy(ratings)

[1, 1, 1, 1, 1]
[1, 2, 2, 2, 1], 5, 0
[1, 3, 3, 3, 1], 3, 0
[1, 4, 4, 4, 1], 1, 0


Probably I'm double-counting, let's only do left-check:

In [27]:
def candy(ratings: list[int]) -> int:
    n = len(ratings)
    candies = [1] * n
    passes = 0
    unchecked = n

    print(f"{candies}, {unchecked}, {passes}")
    while unchecked > 0:
        for i in range(passes + 1, n - passes):
            # Check the left:
            if candies[i] <= candies[i-1] and ratings[i] > ratings[i-1]:
                candies[i] += 1
        unchecked -= 2
        print(f"{candies}, {unchecked}, {passes}")

    return sum(candies)

In [23]:
ratings = [1, 2, 3, 4, 3]
candy(ratings)

[1, 1, 1, 1, 1], 5, 0
[1, 2, 2, 2, 1], 3, 0
[1, 2, 3, 3, 1], 1, 0
[1, 2, 3, 4, 1], -1, 0


Doi, I forgot to check the actual thing we're concerned with before incrementing: whether the _rating_ is higher but count of candies is lower.

In [29]:
ratings = [1,0,2]
candy(ratings)

[1, 1, 1], 3, 0
[1, 1, 2], 1, 0
[1, 1, 2], -1, 0


4

Ah, ok, so we do need a rightward check, because this fails the test case for 0th index being greater than i+1.

In [30]:
def candy(ratings: list[int]) -> int:
    n = len(ratings)
    candies = [1] * n
    passes = 0
    unchecked = n

    print(f"{candies}, {unchecked}, {passes}")
    while unchecked > 0:
        for i in range(passes + 1, n - passes - 1):
            # Check the left:
            if candies[i] <= candies[i-1] and ratings[i] > ratings[i-1]:
                candies[i] += 1
            # Check the right:
            if candies[i] <= candies[i+1] and ratings[i] > ratings[i+1]:
                candies[i] += 1
        unchecked -= 2
        print(f"{candies}, {unchecked}, {passes}")

    return sum(candies)

In [31]:
ratings = [1,0,2]
candy(ratings)

[1, 1, 1], 3, 0
[1, 1, 1], 1, 0
[1, 1, 1], -1, 0


3

Yeah, but now I have the issue of potentially incrementing twice per pass. This needs to be one logical statement:

In [34]:
def candy(ratings: list[int]) -> int:
    n = len(ratings)
    candies = [1] * n
    passes = 0
    unchecked = n

    print(f"{candies}, {unchecked}, {passes}")
    while unchecked > 0:
        for i in range(passes + 1, n - passes - 1):
            # Check the left or right sides simultaneously for an imbalance:
            if ((candies[i] <= candies[i-1] and ratings[i] > ratings[i-1]) or
                (candies[i] <= candies[i+1] and ratings[i] > ratings[i+1])):
                candies[i] += 1
        unchecked -= 2
        print(f"{candies}, {unchecked}, {passes}")

    return sum(candies)

In [35]:
ratings = [1,0,2]
candy(ratings)

[1, 1, 1], 3, 0
[1, 1, 1], 1, 0
[1, 1, 1], -1, 0


3

Oh, duh, I'm not incrementing passes at each iter.

In [36]:
def candy(ratings: list[int]) -> int:
    n = len(ratings)
    candies = [1] * n
    passes = 0
    unchecked = n

    print(f"{candies}, {unchecked}, {passes}")
    while unchecked > 0:
        for i in range(passes + 1, n - passes - 1):
            # Check the left or right sides simultaneously for an imbalance:
            if ((candies[i] <= candies[i-1] and ratings[i] > ratings[i-1]) or
                (candies[i] <= candies[i+1] and ratings[i] > ratings[i+1])):
                candies[i] += 1
        unchecked -= 2
        passes += 1
        print(f"{candies}, {unchecked}, {passes}")

    return sum(candies)

In [37]:
candy(ratings)

[1, 1, 1], 3, 0
[1, 1, 1], 1, 1
[1, 1, 1], -1, 2


3

In [38]:
ratings = [1, 2, 3, 4, 3]
candy(ratings)

[1, 1, 1, 1, 1], 5, 0
[1, 2, 2, 2, 1], 3, 1
[1, 2, 3, 2, 1], 1, 2
[1, 2, 3, 2, 1], -1, 3


9

Ok, this is just busted

---

Ohhh duh, since only relative candies to neighbors is important, we can easily use a greedy approach, since the optimal (minimum) val for a neighbor higher than next neighbor is just that neighbors' value + 1.

Use 1 pass to ensure each index is higher than left neighbor, then a 2nd pass to ensure each index is higher than right neighbor, by termination of the 2nd pass both conditions are true for all indices _and_ at the minimum possible total candies distributed.

In [47]:
def candy(ratings: list[int]) -> int:
    n = len(ratings)
    candies = [1] * n

    print(candies)
    # Ensure each candy is higher than left neighbor:
    for i in range(1, n):
        if candies[i] <= candies[i-1] and ratings[i] > ratings[i-1]:
            candies[i] = candies[i-1] + 1
    print(candies)
    # Ensure each candy is higher than right neighbor:
    for i in range(n-2, -1, -1):
        if candies[i] <= candies[i+1] and ratings[i] > ratings[i+1]:
            candies[i] = candies[i+1] +1
    print(candies)
    return sum(candies)


In [48]:
ratings = [1,0,2]
candy(ratings)

[1, 1, 1]
[1, 1, 2]
[2, 1, 2]


5